In [1]:
import sys
sys.path.append("/raid/kientdt/shared_drive_cv/ocr/kientdt/PaddleOCR2Pytorch")

In [30]:
from tools.program import *
from pytorchocr.data import build_dataloader, set_signal_handlers
import random
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from pytorchocr.postprocess import build_post_process
from pytorchocr.modeling.architectures import build_model

In [3]:
def set_seed(seed=1024):
    random.seed(seed)
    np.random.seed(seed)
def preprocess(is_train=False):
    FLAGS = {
    'profiler_options': None,
    'config': "/raid/kientdt/shared_drive_cv/ocr/kientdt/PaddleOCR/configs/det/ch_PP-OCRv3/vi_PP-OCRv3_det_student_data_v1.yml",
    'opt': {'Global.use_gpu': True, 'Global.epoch_num': 100}
}

    profiler_options = FLAGS.get('profiler_options', None)
    config = load_config(FLAGS['config'])
    config = merge_config(config, FLAGS['opt'])
    profile_dic = {"profiler_options": FLAGS['profiler_options']}
    config = merge_config(config, profile_dic)

    if is_train:
        # save_config
        save_model_dir = config["Global"]["save_model_dir"]
        os.makedirs(save_model_dir, exist_ok=True)
        with open(os.path.join(save_model_dir, "config.yml"), "w") as f:
            yaml.dump(dict(config), f, default_flow_style=False, sort_keys=False)
        log_file = "{}/train.log".format(save_model_dir)
    else:
        log_file = None
    use_gpu = config['Global']['use_gpu']
    alg = config["Architecture"]["algorithm"]
    device = "cuda:0" if use_gpu else "cpu"
    check_device(use_gpu)

    device = torch.cuda.set_device(device)



    return config, device

In [4]:
config, device = preprocess(is_train = True)

In [5]:
global_config = config["Global"]
set_signal_handlers()
seed = config["Global"]["seed"] if "seed" in config["Global"] else 1024
set_seed(seed)
train_dataloader = build_dataloader(config, "Train", device, seed)
if config["Eval"]:
    valid_dataloader = build_dataloader(config, "Eval", device, seed)


In [6]:
for batch in train_dataloader:
    break

In [23]:
# plt.imshow(batch[0][35].numpy().transpose(1, 2, 0))

In [28]:
step_pre_epoch = len(train_dataloader)
post_process_class = build_post_process(config["PostProcess"], global_config)


In [34]:
model = build_model(config["Architecture"])

In [37]:
preds = model(batch[0])

In [38]:
preds = to_float32(preds)


In [39]:
preds

{'maps': tensor([[[[5.2116e-01, 5.0961e-01, 4.9680e-01,  ..., 4.9108e-01,
            4.9937e-01, 5.1685e-01],
           [5.5215e-01, 4.7581e-01, 4.8076e-01,  ..., 4.7473e-01,
            5.1326e-01, 5.3750e-01],
           [6.1516e-01, 5.8495e-01, 5.2106e-01,  ..., 4.9008e-01,
            5.1093e-01, 5.1825e-01],
           ...,
           [5.3327e-01, 5.3244e-01, 5.0378e-01,  ..., 4.8227e-01,
            4.9397e-01, 5.3574e-01],
           [4.1230e-01, 5.2838e-01, 4.6128e-01,  ..., 5.2294e-01,
            5.2513e-01, 5.4198e-01],
           [5.3063e-01, 5.0748e-01, 5.0299e-01,  ..., 5.0892e-01,
            5.2290e-01, 4.8695e-01]],
 
          [[4.6502e-01, 5.0160e-01, 4.9457e-01,  ..., 4.8975e-01,
            5.1876e-01, 5.2386e-01],
           [4.9287e-01, 5.2885e-01, 5.1320e-01,  ..., 5.6217e-01,
            5.1725e-01, 5.1957e-01],
           [4.9971e-01, 4.9910e-01, 5.0501e-01,  ..., 4.9782e-01,
            5.3136e-01, 5.1222e-01],
           ...,
           [5.1738e-01, 5.5837